In [72]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import os
import prepare_flightlog as fl


In [73]:
base_dir = os.path.dirname(os.getcwd())
flightlogs_dir = os.path.join(base_dir, "data", "flightlogs")
files = [f for f in os.listdir(flightlogs_dir)
         if f.endswith(".igc") or f.endswith(".IGC")]
file_path = os.path.join(flightlogs_dir, files[0])
print(f"Reading {file_path}")

Reading /Users/serkan/Documents/Projeler/Paragliding_Flight_Analyses/data/flightlogs/2025-05-10-XNA-170300-01.igc


In [ ]:
df = fl.prepare_data(file_path)
df_new = df[['latitude', 'longitude',
                'gps_altitude_m', 'distance_m', 'speed_km/s',
                'climb_m', 'climb_m(delta)', 'climb_rate_m/s',
                'glide_ratio', 'bearing', 'delta_bearing',
                'elapsed_time', 'delta_time', 'temp',
                'pressure', 'humidity', 'dew_point',
                'wind_speed', 'wind_deg']]





df_new.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,37.326317,37.178167,1100,6.971241,25.096468,-1.0,-8.0,-1.0,6.971241,237,0.0,52.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0
1,37.326283,37.178117,1101,5.772706,20.781740,0.0,-6.0,0.0,0.000000,230,7.0,53.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0
2,37.326267,37.178050,1102,6.191426,22.289135,2.0,-3.0,2.0,3.095713,252,22.0,54.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0
3,37.326233,37.178000,1103,5.772708,20.781748,2.0,0.0,2.0,2.886354,230,22.0,55.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0
4,37.326200,37.177950,1105,5.772709,20.781754,2.0,2.0,2.0,2.886355,230,0.0,56.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0


In [75]:
df_new.describe()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,21113.000000,21113.000000,21113.000000,2.111300e+04,2.111300e+04,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000,21113.000000
mean,37.316584,38.075599,2378.265334,1.633164e+03,2.573087e+03,-0.019893,-0.391702,0.164377,7.204273,117.960025,9.732913,10622.349121,1.001516,30.427615,1011.677450,19.714583,4.563159,1.899723,251.053285
std,0.926586,1.149070,709.900809,9.612143e+04,1.797722e+05,49.253359,110.926826,24.638421,33.805569,77.806860,14.917651,6108.592512,0.068115,1.743952,1.748835,3.473320,1.531253,0.860868,38.134857
min,0.000000,0.000000,743.000000,0.000000e+00,0.000000e+00,-3133.000000,-3129.000000,-626.600000,0.000000,0.000000,0.000000,52.000000,1.000000,26.090000,1009.000000,15.000000,1.700000,0.670000,157.000000
25%,37.323033,37.466383,1875.000000,1.097918e+01,3.952481e+01,-2.000000,-27.000000,-2.000000,2.462910,82.000000,1.000000,5330.000000,1.000000,29.670000,1010.000000,18.000000,4.160000,1.030000,215.000000
50%,37.330050,38.076217,2431.000000,1.328965e+01,4.784273e+01,0.000000,-5.000000,0.000000,5.906249,96.000000,7.000000,10608.000000,1.000000,31.260000,1011.000000,19.000000,4.970000,1.990000,253.000000
75%,37.353067,38.685533,2918.000000,1.488265e+01,5.357747e+01,1.000000,23.000000,1.000000,11.098418,122.000000,11.000000,15918.000000,1.000000,31.590000,1013.000000,22.000000,5.360000,2.570000,288.000000
max,37.397367,39.281350,3782.000000,5.707823e+06,2.051922e+07,3399.000000,3209.000000,2620.000000,2175.489983,355.000000,180.000000,21196.000000,5.000000,32.120000,1014.000000,77.000000,23.520000,5.590000,300.000000


In [76]:
# --- 1. Load saved scaler and Transformer model ---
scaler_path = os.path.join('models', 'scaler.pkl')
model_path  = os.path.join('models', 'transformer_tunned_model.keras')

with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)

model = tf.keras.models.load_model(model_path)
print(f"Loaded Transformer model from '{model_path}'")


# Define feature columns (everything except the target)
feature_cols = [c for c in df_new.columns if c != 'climb_rate_m/s']


Loaded Transformer model from 'models/transformer_tunned_model.keras'


In [77]:

# --- 3. Scale features ---
X_new = df_new[feature_cols].values.astype(np.float32)
X_new_scaled = scaler.transform(X_new)

# --- 4. Create sliding‐window dataset for inference ---
T = 10        # same window length as training
batch_size = 32

predict_ds = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=X_new_scaled,
    targets=None,
    sequence_length=T,
    sequence_stride=1,
    shuffle=False,
    batch_size=batch_size
)

In [78]:
# --- 5. Run predictions ---
preds = model.predict(predict_ds).flatten()

# --- 6. Align predictions with original data indices or timestamps ---
# If your DataFrame has a timestamp column, you can use that instead.
# number of windows = N – T + 1
preds = model.predict(predict_ds).flatten()
# Align preds to df_new by taking index[T-1:]
indices = df_new.index[T-1:]
# Just in case, truncate to the same length:
indices = indices[: len(preds)]

indices = df_new.index[T-1 : T-1 + len(preds)]
df_new.loc[indices, 'predicted_climb_rate_m/s'] = preds

660/660 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
660/660 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/var/folders/ss/5qhyykzn7ws047pz5bmmgw1r0000gn/T/ipykernel_10079/904529400.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[indices, 'predicted_climb_rate_m/s'] = preds


In [79]:
df_new = df_new[df_new['predicted_climb_rate_m/s'].notna()]
df_new = df_new[df_new['climb_rate_m/s'] > 1]
df_new.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,delta_bearing,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg,predicted_climb_rate_m/s
9,37.325783,37.177883,1111,11.196249,40.306496,3.0,7.0,3.0,3.732083,172,1.0,61.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.151743
10,37.325700,37.177900,1114,9.365869,33.717130,3.0,11.0,3.0,3.121956,170,2.0,62.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.217597
14,37.325450,37.177833,1118,7.966945,28.681003,2.0,15.0,2.0,3.983473,201,10.0,66.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.213835
15,37.325400,37.177800,1120,6.286632,22.631876,2.0,18.0,2.0,3.143316,207,6.0,67.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.164799
16,37.325350,37.177767,1122,6.286633,22.631879,2.0,20.0,2.0,3.143317,207,0.0,68.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.231754


In [80]:
y_true = df_new['climb_rate_m/s'].values
y_pred = df_new['predicted_climb_rate_m/s'].values

# 2) Pearson korelasyon (−1…1 → map 0…1)
pearson_r = np.corrcoef(y_true, y_pred)[0,1]
pearson_sim = (pearson_r + 1) / 2
pearson_percent = pearson_sim * 100
print(f"Pearson similarity: {pearson_percent:.2f}%")

Pearson similarity: 52.22%


In [81]:
# Orijinal y_true, y_pred
y_true_centered = y_true - np.mean(y_true)
y_pred_centered = y_pred - np.mean(y_pred)

# Normalize
y_true_n = y_true_centered / np.linalg.norm(y_true_centered)
y_pred_n = y_pred_centered / np.linalg.norm(y_pred_centered)

cos_sim2 = cosine_similarity(
    y_true_n.reshape(1,-1),
    y_pred_n.reshape(1,-1)
)[0,0]
print(f"Centered Cosine: {cos_sim2*100:.2f}%")

Centered Cosine: 4.44%


In [82]:
# 1) Gerçek ve tahmin verilerini oku
y_true = df_new['climb_rate_m/s'].values
y_pred = df_new['predicted_climb_rate_m/s'].values

# 2) Parametreler
E_max       = 2.0    # 2 m/s üzerindeki hataları sıfır kabul et

# 3) Zaman adımı puanları
errors = y_pred - y_true
print(errors)
score_t = np.clip(1 - (errors / E_max), 0, 1) * 100


# Her adım en fazla 100 olmalı
flight_score = score_t.mean()

print(f"Bu uçuş için performans skoru: {flight_score:.2f}/100")



[-1.84825706 -1.78240311 -0.78616524 ... -0.81671166 -0.81106949
 -0.90634239]
Bu uçuş için performans skoru: 85.42/100


In [85]:
def calc_score(row):
    return row['predicted_climb_rate_m/s'] - row['climb_rate_m/s']

df_new["score"] = df_new.apply(calc_score, axis=1)
df_new.head()

,latitude,longitude,gps_altitude_m,distance_m,speed_km/s,climb_m,climb_m(delta),climb_rate_m/s,glide_ratio,bearing,...,elapsed_time,delta_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg,predicted_climb_rate_m/s,score
9,37.325783,37.177883,1111,11.196249,40.306496,3.0,7.0,3.0,3.732083,172,...,61.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.151743,-1.848257
10,37.325700,37.177900,1114,9.365869,33.717130,3.0,11.0,3.0,3.121956,170,...,62.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.217597,-1.782403
14,37.325450,37.177833,1118,7.966945,28.681003,2.0,15.0,2.0,3.983473,201,...,66.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.213835,-0.786165
15,37.325400,37.177800,1120,6.286632,22.631876,2.0,18.0,2.0,3.143316,207,...,67.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.164799,-0.835201
16,37.325350,37.177767,1122,6.286633,22.631879,2.0,20.0,2.0,3.143317,207,...,68.0,1.0,26.09,1014.0,26.0,5.11,0.71,264.0,1.231754,-0.768246
